### 模式匹配
“模式匹配”是一种测试表达式是否具有特定特征的方法。 C# 模式匹配提供更简洁的语法，用于测试表达式并在表达式匹配时采取措施。 “is 表达式”目前支持通过模式匹配测试表达式并有条件地声明该表达式结果。 “switch 表达式”允许你根据表达式的首次匹配模式执行操作。 这两个表达式支持丰富的模式词汇。

#### Null 检查

In [2]:
int? maybe = 12;

if (maybe is int number)
{
    Console.WriteLine($"The nullable int 'maybe' has the value {number}");
}
else
{
    Console.WriteLine("The nullable int 'maybe' doesn't hold a value");
}
//变量 number 仅在 if 子句的 true 部分可供访问和分配。 如果尝试在 else 子句或 if 程序块后等其他位置访问，编译器将出错。 其次，由于不使用 == 运算符，因此当类型重载 == 运算符时，此模式有效。

The nullable int 'maybe' has the value 12


In [6]:
string? message = "hello world";

if (message is not null)
{
    Console.WriteLine(message);
}
//not 为一种逻辑模式，在否定模式不匹配时与该模式匹配。

hello world


#### 类型测试
模式匹配的另一种常见用途是测试变量是否与给定类型匹配。 例如，以下代码测试变量是否为非 null 并实现 System.Collections.Generic.IList<T> 接口。 如果是，它将使用该列表中的 ICollection<T>.Count 属性来查找中间索引。 不管变量的编译时类型如何，声明模式均与 null 值不匹配。 除了防范未实现 IList 的类型之外，以下代码还可防范 null。

In [7]:
public static T MidPoint<T>(IEnumerable<T> sequence)
{
    if (sequence is IList<T> list)
    {
        return list[list.Count / 2];
    }
    else if (sequence is null)
    {
        throw new ArgumentNullException(nameof(sequence), "Sequence can't be null.");
    }
    else
    {
        int halfLength = sequence.Count() / 2 - 1;
        if (halfLength < 0) halfLength = 0;
        return sequence.Skip(halfLength).First();
    }
}

#### 比较离散值
你还可以通过测试变量找到特定值的匹配项。 在以下代码演示的示例中，你针对枚举中声明的所有可能值进行数值测试：

In [ ]:
public State PerformOperation(Operation command) =>
   command switch
   {
       Operation.SystemTest => RunDiagnostics(),
       Operation.Start => StartSystem(),
       Operation.Stop => StopSystem(),
       Operation.Reset => ResetToReady(),
       _ => throw new ArgumentException("Invalid enum value for command", nameof(command)),
   };

In [19]:
int num = 5;

string message = num switch
{
    1 => "One",
    2 => "Two",
    3 => "Three",
    _ => "Other"
};

Console.WriteLine(message);


Other


#### 关系模式
你可以使用关系模式测试如何将数值与常量进行比较。 例如，以下代码基于华氏温度返回水源状态：

In [32]:
string WaterState(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        (> 32) and (< 212) => "液态",
        < 32 => "固态",
        > 212 => "气态",
        32 => "solid/liquid transition",
        212 => "liquid / gas transition",
    };

WaterState(250).Display();

气态

In [33]:
string WaterState2(int tempInFahrenheit) =>
    tempInFahrenheit switch
    {
        < 32 => "solid",
        32 => "solid/liquid transition",
        < 212 => "liquid",
        212 => "liquid / gas transition",
        _ => "gas",
};

#### 多个输入
可以写入检查一个对象的多个属性的模式。 请考虑以下 Order 记录：

In [42]:
public record Order(int Items, decimal Cost);
public decimal CalculateDiscount(Order order) =>
    order switch
    {
        { Items: > 10, Cost: > 1000.00m } => 0.10m,
        { Items: > 5, Cost: > 500.00m } => 0.05m,
        { Cost: > 250.00m } => 0.02m,
        null => throw new ArgumentNullException(nameof(order), "Can't calculate discount on null order"),
        var someObject => 0m,
    };

var order1 = new Order(5, 500.00m);
CalculateDiscount(order1).Display(); 
var order2 = new Order(2, 150.00m);
CalculateDiscount(order2).Display(); 


0.02

0

#### 列表模式
可以使用列表模式检查列表或数组中的元素。 列表模式提供了一种方法，将模式应用于序列的任何元素。 此外，还可以应用弃元模式 (_) 来匹配任何元素，或者应用切片模式来匹配零个或多个元素。
当数据不遵循常规结构时，列表模式是一个有价值的工具。 可以使用模式匹配来测试数据的形状和值，而不是将其转换为一组对象。

04-01-2020, DEPOSIT,    Initial deposit,            2250.00
04-15-2020, DEPOSIT,    Refund,                      125.65
04-18-2020, DEPOSIT,    Paycheck,                    825.65
04-22-2020, WITHDRAWAL, Debit,           Groceries,  255.73
05-01-2020, WITHDRAWAL, #1102,           Rent, apt, 2100.00
05-02-2020, INTEREST,                                  0.65
05-07-2020, WITHDRAWAL, Debit,           Movies,      12.57
04-15-2020, FEE,                                       5.55

In [ ]:
decimal balance = 0m;
foreach (string[] transaction in ReadRecords())
{
    balance += transaction switch
    {
        [_, "DEPOSIT", _, var amount]     => decimal.Parse(amount),
        [_, "WITHDRAWAL", .., var amount] => -decimal.Parse(amount),
        [_, "INTEREST", var amount]       => decimal.Parse(amount),
        [_, "FEE", var fee]               => -decimal.Parse(fee),
        _                                 => throw new InvalidOperationException($"Record {string.Join(", ", transaction)} is not in the expected format!"),
    };
    Console.WriteLine($"Record: {string.Join(", ", transaction)}, New balance: {balance:C}");
}